In [11]:
import pandas as pd
from sklearn.utils import shuffle
from pymorphy2 import MorphAnalyzer
import re

In [12]:
positive_tweets = pd.read_csv('data/positive.csv', sep = ';', header = None)
negative_tweets = pd.read_csv('data/negative.csv', sep = ';', header = None)
dataset = pd.concat([positive_tweets, negative_tweets])[[3, 4]]
dataset.columns = ['text', 'label']

In [13]:
dataset = dataset.sample(frac = 1, random_state = 42).reset_index(drop = True)
dataset.head(5)

,text,label
0,мыс на меня обиделась:(\nя ей даже ничего не с...,-1
1,"аааааааааааааааааааа,не хочу на работу :(",-1
2,"У меня какой-то особенный вид ушей! :D, некото...",1
3,@simonovkon он неплохой человек в жизни. Я ра...,-1
4,"RT @Darina_Lo: Домааааа\nЕхали на такси, пели ...",1


In [14]:
analyzer = MorphAnalyzer()

In [15]:
def clean(txt, use_lemmatization = False):
    txt = txt.lower()
    txt = re.sub(r'\s+', ' ', txt)
    letters = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '
    analyzer = MorphAnalyzer()
    result = ''
    for letter in txt:
        if letter in letters:
            result += letter
    if use_lemmatization:
        temp = []
        for word in result.split():
            temp.append(analyzer.parse(word)[0].normal_form)
        result = ' '.join(temp)
    return result

In [ ]:
dataset_with_lemmatization = dataset['text'].apply(lambda x : clean(x, use_lemmatization = True))

In [ ]:
dataset_without_lemmatization = dataset['text'].apply(lambda x : clean(x, use_lemmatization = False))

In [ ]:
dataset_with_lemmatization.to_csv("data/data_with_lemmatization.csv")
dataset_without_lemmatization.to_csv("data/data_without_lemmatization.csv")

In [76]:
dataset_with_lemmatization = pd.read_csv("data/data_with_lemmatization.csv", header = None)
dataset_without_lemmatization = pd.read_csv("data/data_without_lemmatization.csv", header = None)
dataset_with_lemmatization = dataset_with_lemmatization[[1,]]
dataset_without_lemmatization = dataset_without_lemmatization[[1,]]
dataset_with_lemmatization['label'] = dataset['label']
dataset_without_lemmatization['label'] = dataset['label']
dataset_with_lemmatization.columns, dataset_without_lemmatization.columns = ['text', 'label'], ['text', 'label']

In [77]:
dataset_without_lemmatization.head()

,text,label
0,мыс на меня обиделась я ей даже ничего не сделала,-1
1,аааааааааааааааааааане хочу на работу,-1
2,у меня какойто особенный вид ушей некоторые в...,1
3,он неплохой человек в жизни я работала в шоуб...,-1
4,домааааа ехали на такси пели песни отдыхали ...,1


In [78]:
from gensim.models import Word2Vec
from catboost import CatBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import ShuffleSplit
from sklearn.model_selection import train_test_split
import numpy as np

In [79]:
dataset_with_lemmatization = dataset_with_lemmatization.dropna()
dataset_without_lemmatization = dataset_without_lemmatization.dropna()
dataset_without_lemmatization.head()

,text,label
0,мыс на меня обиделась я ей даже ничего не сделала,-1
1,аааааааааааааааааааане хочу на работу,-1
2,у меня какойто особенный вид ушей некоторые в...,1
3,он неплохой человек в жизни я работала в шоуб...,-1
4,домааааа ехали на такси пели песни отдыхали ...,1


In [60]:
model = Word2Vec(dataset_with_lemmatization['text'])

In [80]:
n_gram = (1, 3)
vectorizer = TfidfVectorizer(analyzer = "word", ngram_range=n_gram)
datasets = [dataset_with_lemmatization, dataset_without_lemmatization]
parameters = {
        'loss': ('log', 'hinge'),
        'penalty': ['none', 'l1', 'l2', 'elasticnet'],
        'alpha': [0.001, 0.0001, 0.00001, 0.000001]
    }
for data in datasets:
    X = vectorizer.fit_transform(data['text'])
    y = data['label']
    cv = ShuffleSplit(len(y), n_iter = 5, test_size = 0.3, random_state = 0)
    gs = GridSearchCV(SGDClassifier(), parameters, cv = cv)
    gs.fit(X, y)
    print('----------------------')
    print('Score of GS : {}'.format(gs.best_score_))
    print('Parameters : {}'.format(gs.best_params_))
    print('----------------------')

----------------------
Score of GS : 0.7583564784199621
Parameters : {'alpha': 1e-06, 'loss': 'log', 'penalty': 'l2'}
----------------------
----------------------
Score of GS : 0.7581549132268446
Parameters : {'alpha': 1e-06, 'loss': 'log', 'penalty': 'l2'}
----------------------
